In [9]:
import time
import csv
import os
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [10]:
from itertools import zip_longest

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [18]:
def load_data():
    data = []
    with open("./data/suncream_list.csv") as fr:
        reader = csv.DictReader(fr)
        for row in reader:
            data.append(row)
    return data[200:300]

In [19]:
def write_data(data):
    file_path = "./data/suncream_reviews_score_200.csv"
    file_exists = os.path.isfile("./data/suncream_reviews_score_200.csv")
    
    with open(file_path, "a", newline='', encoding='utf-8') as fw:
        writer = csv.DictWriter(fw, fieldnames=["page", "product_name", "star", "title", "review", "skin_type"])
        
        # 파일이 존재하지 않으면 헤더를 작성합니다.
        if not file_exists:
            writer.writeheader()
        
        for row in data:
            writer.writerow(row)

In [6]:
def test(url):
    driver = webdriver.Chrome()
    
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]'))).click()
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    star_tags = soup.find_all("div", class_ = "score_area")

    for star_tag in star_tags:
        star_texts = [star.text.strip() for star in star_tag.find_all("span")][0]
        break

    return star_texts

In [14]:
test("https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000199740&dispCatNo=100000100110004&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%84%A0%EC%BF%A0%EC%85%98_%EC%A0%84%EC%B2%B4__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1")

'5점만점에 5점'

In [20]:
def crawl_parse_review_html_write_data(url):
    
    driver = webdriver.Chrome()
    
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]'))).click()
        time.sleep(3)

        for page_num in range(1, 200):
            parse_review_text_list = []
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')

            product_name_tag = soup.find("p", class_="prd_name")
            product_name = product_name_tag.text if product_name_tag else []

            user_clrfix_tags = soup.find_all("div", class_="user clrfix")
            review_tags = soup.find_all("div", class_="txt_inner")
            title_tags = soup.find_all("div", class_="poll_sample")
            star_tags = soup.find_all("div", class_ = "score_area")

            combined_list = list(zip_longest(title_tags, review_tags, user_clrfix_tags, star_tags, fillvalue=None))

            for title_tag, review_tag, user_clrfix_tag, star_tag in combined_list:
                review_text = review_tag.text if review_tag else None
                title_text = [tag.text.strip() for tag in title_tag.find_all("span")[1::2]] if title_tag else None
                span_text = [span.text.strip() for span in user_clrfix_tag.find_all("span")[1:]] if user_clrfix_tag else None
                star_text = [star.text.strip() for star in star_tag.find_all("span")][0]

                review_data = {
                    "page": page_num,
                    "product_name": product_name,
                    "star" : star_text,
                    "title": title_text,
                    "review": review_text,
                    "skin_type": span_text
                }
                parse_review_text_list.append(review_data)

            write_data(parse_review_text_list)

            try:
                next_button = driver.find_element(By.XPATH, f"//a[@data-page-no='{page_num + 1}']")
                next_button.click()
                time.sleep(3)
            except NoSuchElementException:
                print(f"Page {page_num} is the last page. \n ")
                break

    finally:
        driver.quit()
    
    return parse_review_text_list

In [21]:
if __name__ == '__main__':
    data = load_data()
    parse_review_list = []
    
    for i, review in enumerate(data):
        url = review["product_link"]
        try:
            parse_review = crawl_parse_review_html_write_data(url)
        except:
            continue
        print(f"{i}번 제품 끝")
        

Page 4 is the last page. 
 
0번 제품 끝
Page 1 is the last page. 
 
1번 제품 끝
Page 4 is the last page. 
 
2번 제품 끝
Page 13 is the last page. 
 
3번 제품 끝
Page 73 is the last page. 
 
4번 제품 끝
Page 8 is the last page. 
 
5번 제품 끝
Page 8 is the last page. 
 
6번 제품 끝
Page 1 is the last page. 
 
7번 제품 끝
Page 10 is the last page. 
 
8번 제품 끝
Page 100 is the last page. 
 
9번 제품 끝
Page 19 is the last page. 
 
10번 제품 끝
Page 1 is the last page. 
 
11번 제품 끝
Page 50 is the last page. 
 
12번 제품 끝
Page 2 is the last page. 
 
13번 제품 끝
Page 11 is the last page. 
 
14번 제품 끝
Page 17 is the last page. 
 
15번 제품 끝
Page 9 is the last page. 
 
16번 제품 끝
Page 5 is the last page. 
 
17번 제품 끝
Page 8 is the last page. 
 
18번 제품 끝
Page 5 is the last page. 
 
19번 제품 끝
Page 16 is the last page. 
 
20번 제품 끝
Page 45 is the last page. 
 
21번 제품 끝
Page 2 is the last page. 
 
22번 제품 끝
Page 32 is the last page. 
 
23번 제품 끝
Page 22 is the last page. 
 
24번 제품 끝
Page 15 is the last page. 
 
25번 제품 끝
Page 9 is the last page. 
 
26번 제품